In [ ]:
from data_utils import *
import IPython.display as ipd
import torch
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from infer_utils import preprocess_input
hps = utils.get_hparams_from_file("configs/ljs_base.json")

net_g = SynthesizerTrn(
  len(symbols),
  hps.data.filter_length // 2 + 1,
  hps.data.hop_length,
  hps.data.sampling_rate,
  hps.train.segment_size // hps.data.hop_length,
  **hps.model)

In [ ]:

def infer(inp,length_scale=1):
  item = preprocess_input(inp)
  ph_token = torch.LongTensor(item["ph_token"]).unsqueeze(0)
  ph_len = torch.LongTensor([item["ph_len"]])
  pitch_midi = torch.LongTensor(item["pitch_midi"]).unsqueeze(0)
  midi_dur = torch.FloatTensor(item["midi_dur"]).unsqueeze(0)
  y_hat, mask, xx = net_g.infer(ph_token, ph_len, pitch_midi, midi_dur,max_len=1000,length_scale=length_scale)
  audio = y_hat[0,0].data.float().numpy()
  ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

In [ ]:
_ = utils.load_checkpoint("G_236000.pth", net_g, None)

In [ ]:
inp = {
  'text': "你 说 你 不 SP 懂 为 何 在 这 时 牵 手 AP",
  'notes': "D#4/Eb4 | D#4/Eb4 | D#4/Eb4  | D#4/Eb4 | rest | D#4/Eb4 | D4 | D4 | D4 | D#4/Eb4 | F4 | D#4/Eb4 | D4 | rest",
  'notes_duration': "0.113740 | 0.329060 | 0.287950 | 0.133480 | 0.150900 | 0.484730 | 0.242010 | 0.180820 | 0.343570 | 0.152050 | 0.266720 | 0.280310 | 0.633300 | 0.444590",
  'input_type': 'word'
}

infer(inp,1.1)

In [ ]:
inp = {
  "note_seq": "C4 C4 D4 D4 D4 D4 D4 D4 rest D4 D4 A#3/Bb3 A#3/Bb3 rest A#3/Bb3 A#3/Bb3 A#3/Bb3 A#3/Bb3 rest A#3/Bb3 A#3/Bb3 A#3/Bb3 A#3/Bb3 D#4/Eb4 D#4/Eb4 rest",
  "note_dur_seq": "0.232140 0.232140 0.160180 0.160180 0.363150 0.363150 0.161340 0.161340 0.074290 0.280500 0.280500 0.502820 0.502820 0.211830 0.301390 0.301390 0.116070 0.116070 0.122460 0.252690 0.252690 0.502010 0.502010 0.475340 0.475340 0.335450",
  "ph_seq": "y v l in sh i l e SP t ian k ong AP h ui d e SP g eng j iang j iu AP",
  "input_type": "phoneme"
}
infer(inp,1.1)

In [1]:
from infer_utils import preprocess_input

In [6]:
inp = {
  'text': "摸多",
  'notes': "D#4/Eb4",
  'notes_duration': "0.113740",
  'input_type': 'word'
}
preprocess_input(inp)

The number of words does't match the number of notes' windows.  You should split the note(s) for each word by | mark.
['m o', 'd uo'] ['D#4/Eb4'] ['0.113740']
2 1 1
==========> Preprocess_word_level or phone_level input wrong.
